# 🧱 Mechano-Velocity: Notebook 02 - Mechanotyping

**Calculate the ECM Resistance Field**

This notebook implements the core mechanotyping algorithm:
1. Extract collagen and ECM gene signatures
2. Calculate raw density using the physics equation
3. Normalize to resistance probability [0, 1]
4. Validate against H&E histology
5. Run virtual drug simulation

---

## The Physics Equation

$$D_i = (\alpha \cdot COL1A1 + \alpha \cdot COL1A2) \times (1 + \beta \cdot LOX) - (\gamma \cdot MMP9)$$

$$R_i = \frac{1}{1 + e^{-(D_i - \mu)}}$$

Where:
- $D_i$: Raw density score for spot $i$
- $R_i$: Normalized resistance probability [0, 1]
- $\alpha, \beta, \gamma$: Hyperparameters
- $\mu$: Centering parameter (dataset mean)

## 1. Setup

In [ ]:
# Check if running in Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    %cd /content/mechano-velocity
    !pip install -q scanpy squidpy

In [ ]:
# Core imports
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import warnings
warnings.filterwarnings('ignore')

sc.settings.verbosity = 2
sc.settings.set_figure_params(dpi=100, frameon=False, figsize=(8, 8))

In [ ]:
# Import project modules
from pathlib import Path
import sys

PROJECT_ROOT = Path('.').resolve()
sys.path.insert(0, str(PROJECT_ROOT))

from mechano_velocity import Config, Mechanotyper, Visualizer
from mechano_velocity.config import GeneSignature, MechanotypingParams

## 2. Load Preprocessed Data

In [ ]:
# Load configuration
config = Config()
config.output_dir = PROJECT_ROOT / "output"

# Load preprocessed data from previous notebook
adata_path = config.output_dir / 'preprocessed_adata.h5ad'

if adata_path.exists():
    adata = sc.read_h5ad(adata_path)
    print(f"Loaded: {adata.shape}")
else:
    raise FileNotFoundError(f"Please run 01_Preprocessing.ipynb first. Expected: {adata_path}")

In [ ]:
# Quick overview
print("AnnData structure:")
print(f"  Spots: {adata.n_obs}")
print(f"  Genes: {adata.n_vars}")
print(f"  Layers: {list(adata.layers.keys())}")

## 3. Explore Gene Signatures

Let's understand the biological "players" before calculating resistance.

In [ ]:
# Define gene signature
gene_sig = GeneSignature()

print("Gene Signature for Mechanotyping:")
print("\n🧱 CONSTRUCTION TEAM (Barrier Builders):")
print(f"  Collagen (The Bricks): {gene_sig.collagen_genes}")
print(f"  Cross-linkers (The Cement): {gene_sig.crosslinker_genes}")
print(f"  Scaffold (The Frame): {gene_sig.scaffold_genes}")

print("\n🔨 DEMOLITION TEAM (Tunnel Borers):")
print(f"  Degradation (The Drills): {gene_sig.degradation_genes}")

print("\n🏃 TRAVELERS (Mobile Agents):")
print(f"  T-cell markers: {gene_sig.tcell_markers}")
print(f"  Tumor markers: {gene_sig.tumor_markers}")

In [ ]:
# Check availability
all_genes = gene_sig.all_genes
available = [g for g in all_genes if g in adata.var_names]
missing = [g for g in all_genes if g not in adata.var_names]

print(f"\n✅ Available: {len(available)}/{len(all_genes)} genes")
print(f"Available: {available}")
if missing:
    print(f"\n❌ Missing: {missing}")

In [ ]:
# Visualize individual gene expression spatially
plot_genes = [g for g in ['COL1A1', 'COL1A2', 'LOX', 'MMP9'] if g in adata.var_names]

if plot_genes:
    n_genes = len(plot_genes)
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    axes = axes.flatten()
    
    for i, gene in enumerate(plot_genes):
        if i < len(axes):
            sc.pl.spatial(adata, color=gene, ax=axes[i], show=False, 
                         title=f"{gene} Expression", cmap='Reds')
    
    # Hide unused axes
    for i in range(len(plot_genes), len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.savefig(config.output_dir / 'mechanotyping_genes.png', dpi=150)
    plt.show()

## 4. Configure Hyperparameters

These control the physics equation weights.

In [ ]:
# Display current parameters
params = config.mechanotyping

print("Mechanotyping Hyperparameters:")
print(f"  α (collagen weight): {params.alpha}")
print(f"  β (LOX multiplier): {params.beta}")
print(f"  γ (MMP penalty): {params.gamma}")
print(f"  KNN smoothing: {params.knn_smoothing} neighbors")
print(f"  Wall threshold: R > {params.wall_threshold}")
print(f"  Fluid threshold: R < {params.fluid_threshold}")

In [ ]:
# You can modify parameters here
# config.mechanotyping.alpha = 1.0
# config.mechanotyping.beta = 0.5
# config.mechanotyping.gamma = 0.8

## 5. Calculate Resistance Field

In [ ]:
# Initialize mechanotyper
mechanotyper = Mechanotyper(config)

# Calculate resistance
resistance = mechanotyper.calculate_resistance(
    adata,
    smooth=True,           # Apply KNN smoothing for zero-inflation
    store_in_adata=True    # Store results in adata.obs
)

In [ ]:
# View stored results
print("\nNew columns in adata.obs:")
new_cols = ['raw_density', 'resistance', 'resistance_category']
for col in new_cols:
    if col in adata.obs.columns:
        print(f"  {col}: {adata.obs[col].dtype}")

# Show gene expression columns
expr_cols = [c for c in adata.obs.columns if c.startswith('expr_')]
if expr_cols:
    print(f"  Gene expressions: {expr_cols}")

In [ ]:
# Statistical summary
print("\nResistance Statistics:")
print(adata.obs['resistance'].describe())

## 6. Visualize Resistance Field

In [ ]:
# Initialize visualizer
viz = Visualizer(config)

In [ ]:
# Plot resistance heatmap
fig = viz.plot_resistance_heatmap(
    adata,
    show_image=True,
    spot_size=50,
    alpha=0.8,
    title="ECM Resistance Field\n(Red=Wall, Blue=Fluid)",
    save_path=config.output_dir / 'resistance_map.png'
)
plt.show()

In [ ]:
# Distribution of resistance values
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(adata.obs['resistance'], bins=50, color='steelblue', edgecolor='white', alpha=0.7)
axes[0].axvline(x=params.wall_threshold, color='red', linestyle='--', linewidth=2, label=f'Wall ({params.wall_threshold})')
axes[0].axvline(x=params.fluid_threshold, color='blue', linestyle='--', linewidth=2, label=f'Fluid ({params.fluid_threshold})')
axes[0].set_xlabel('Resistance Score', fontsize=12)
axes[0].set_ylabel('Number of Spots', fontsize=12)
axes[0].set_title('Distribution of Resistance Values', fontsize=14)
axes[0].legend()

# Category pie chart
category_counts = adata.obs['resistance_category'].value_counts()
colors = {'wall': 'firebrick', 'normal': 'gray', 'fluid': 'steelblue'}
axes[1].pie(
    category_counts.values, 
    labels=category_counts.index, 
    colors=[colors.get(c, 'gray') for c in category_counts.index],
    autopct='%1.1f%%',
    startangle=90
)
axes[1].set_title('Resistance Categories', fontsize=14)

plt.tight_layout()
plt.savefig(config.output_dir / 'resistance_distribution.png', dpi=150)
plt.show()

In [ ]:
# Spatial view by category
fig, ax = plt.subplots(figsize=(10, 10))
sc.pl.spatial(
    adata, 
    color='resistance_category',
    palette={'wall': 'firebrick', 'normal': 'lightgray', 'fluid': 'steelblue'},
    ax=ax,
    show=False,
    title="Resistance Categories\n(Wall / Normal / Fluid)"
)
plt.tight_layout()
plt.savefig(config.output_dir / 'resistance_categories.png', dpi=150)
plt.show()

## 7. Validation: Compare with H&E Image

The **pink eosinophilic streaks** in H&E staining represent collagen fibers.
Our resistance map should align with these regions.

In [ ]:
# Side-by-side comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# H&E image only
sc.pl.spatial(adata, ax=axes[0], show=False, title="H&E Staining\n(Pink = Collagen)")

# Resistance overlay
sc.pl.spatial(adata, color='resistance', ax=axes[1], show=False, 
              title="Resistance Field Overlay", cmap='RdBu_r', alpha_img=0.5)

plt.tight_layout()
plt.savefig(config.output_dir / 'validation_overlay.png', dpi=150)
plt.show()

print("\n📋 VALIDATION CHECKLIST:")
print("  □ Do red (high resistance) regions match pink fibrous areas?")
print("  □ Are blue (low resistance) regions in loose stroma/fat?")
print("  □ Is the tumor core surrounded by a resistance ring?")

## 8. Virtual Drug Simulation

**Hypothesis:** If we inhibit LOX (cross-linking enzyme), the wall should become weaker.

This validates that our model responds to therapeutic variables.

In [ ]:
# Store original resistance
original_resistance = adata.obs['resistance'].values.copy()

In [ ]:
# Simulate LOX inhibitor (reduce LOX expression to 0)
simulated_resistance = mechanotyper.simulate_drug(
    adata,
    target_gene='LOX',
    reduction_factor=0.0  # Complete knockout
)

In [ ]:
# Visualize drug effect
fig = viz.plot_drug_simulation(
    adata,
    original_resistance=original_resistance,
    simulated_resistance=simulated_resistance,
    drug_name="LOX Inhibitor",
    save_path=config.output_dir / 'drug_simulation_lox.png'
)
plt.show()

In [ ]:
# Quantify drug effect
print("\n💊 DRUG SIMULATION SUMMARY:")
print(f"  Original mean resistance: {original_resistance.mean():.4f}")
print(f"  Simulated mean resistance: {simulated_resistance.mean():.4f}")
print(f"  Change: {(simulated_resistance.mean() - original_resistance.mean()) * 100:.2f}%")

# Count category changes
orig_walls = (original_resistance > params.wall_threshold).sum()
sim_walls = (simulated_resistance > params.wall_threshold).sum()
print(f"\n  Wall spots (before): {orig_walls}")
print(f"  Wall spots (after): {sim_walls}")
print(f"  Walls dissolved: {orig_walls - sim_walls}")

## 9. Correlation Analysis

In [ ]:
# Correlate resistance with cluster identities
if 'leiden' in adata.obs.columns:
    cluster_resistance = adata.obs.groupby('leiden')['resistance'].mean().sort_values(ascending=False)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    cluster_resistance.plot(kind='bar', ax=ax, color='steelblue', edgecolor='white')
    ax.axhline(y=params.wall_threshold, color='red', linestyle='--', label='Wall threshold')
    ax.set_xlabel('Cluster', fontsize=12)
    ax.set_ylabel('Mean Resistance', fontsize=12)
    ax.set_title('Mean Resistance by Cluster', fontsize=14)
    ax.legend()
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.savefig(config.output_dir / 'resistance_by_cluster.png', dpi=150)
    plt.show()
    
    print("\nCluster Resistance Ranking (High → Low):")
    for cluster, res in cluster_resistance.items():
        status = "⬛ WALL" if res > params.wall_threshold else "" 
        print(f"  Cluster {cluster}: {res:.4f} {status}")

## 10. Save Results

In [ ]:
# Save updated AnnData with resistance
output_path = config.output_dir / 'mechanotyped_adata.h5ad'
adata.write_h5ad(output_path)
print(f"Saved mechanotyped data to: {output_path}")

In [ ]:
# Export resistance values to CSV
resistance_df = adata.obs[['resistance', 'raw_density', 'resistance_category']].copy()
resistance_df['x'] = adata.obsm['spatial'][:, 0]
resistance_df['y'] = adata.obsm['spatial'][:, 1]

csv_path = config.output_dir / 'resistance_values.csv'
resistance_df.to_csv(csv_path)
print(f"Exported to: {csv_path}")

## Summary

✅ Extracted collagen and ECM gene signatures  
✅ Calculated raw density using the physics equation  
✅ Normalized resistance to [0, 1] range  
✅ Applied KNN smoothing for zero-inflation  
✅ Visualized resistance heatmap  
✅ Validated against H&E histology  
✅ Simulated LOX inhibitor drug effect  
✅ Saved mechanotyped data  

**Next: Run `03_Graph_Simulation.ipynb` to build the spatial graph and correct velocities.**